# Plot ROI Contours and Corresponding Whole-Session Activity Traces

What does this script do
------------------------------------

Plots the mean motion-corrected projection image with select ROI contours displayed. Also plots each ROI's corresponding neuropil-corrected dF/F activity trace across the whole session.

Requires SIMA to have run through the loaded dataset. I have a sample dataset (resides in the sample_data\VJ_OFCVTA_7_260_D6_snippit folder) that can be readily run through this script for your reference of file/folder structure and expected output.

How to run this code
------------------------------------

In this jupyter notebook, First find the code block with the comment header called USER-DEFINED VARIABLES. Edit the variables according to your data and output preferences. Then just run all cells in order (shift + enter; or in the dropdown menu: Kernel->Resart & Run All).

Parameters
------------------------------------

__fname_signals__ : string 

    Name of file that contains roi activity traces. Must include full file name with extension. Accepted file types: .npy, .csv. IMPORTANT: data dimensions should be rois (y) by samples/time (x)

__fdir__ : string 

    Root file directory containing the suite2p folder. IMPORTANT Note: leave off the last backslash, and include the letter r in front of string (treats the contents as a raw string). For example: r'C:\Users\my_user\analyze_sessions'

__rois_to_plot__ can be one of the following: 
1) A list of select rois \
2) An integer (n) indicating n first rois to plot \
3) None which plots all valid ROIs 

In [ ]:
import os
import numpy as np
import h5py
import pandas as pd
import pickle
import numpy as np
from PIL import Image

import matplotlib.pyplot as plt
plt.rcParams['text.usetex'] = False
plt.rcParams['text.latex.unicode'] = False
import matplotlib
# important for text to be detecting when importing saved figures into illustrator
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42
plt.rcParams["font.family"] = "Arial"

import utils

In [ ]:
# USER DEFINED VARIABLES
fs = 5
proj_type = 'std' # can be 'std', 'mean', 'max'

fname_signals = 'VJ_OFCVTA_7_260_D6_snippit_neuropil_corrected_signals_15_50_beta_0.8.npy'
fdir = os.path.abspath('./sample_data/VJ_OFCVTA_7_260_D6_snippit') # NOTE: the root folder name must match the basename of the _sima_masks.npy file

"""
define number of ROIs to visualize

can be: 
1) a list of select rois, 
2) an integer (n) indicating n first rois to plot, or 
3) None which plots all valid ROIs
""" 
rois_to_plot =  np.arange(5) #[0,2,3,6] 

In [ ]:
fname = os.path.split(fdir)[-1]

proj_img_path = os.path.join(fdir, f'{fname}_output_images\std_img.tif')
sima_h5_path = os.path.join(fdir, f'{fname}_sima_mc.h5')
sima_mask_path = os.path.join(fdir, f'{fname}_sima_masks.npy')

fig_save_dir = os.path.join(fdir, 'figs')
if not os.path.exists(fig_save_dir):
    os.mkdir(fig_save_dir)


# SIMA 

In [ ]:
roi_signal_sima = np.load(os.path.join(fdir, fname_signals))
signal_dff = np.apply_along_axis(utils.calc_dff, 1, roi_signal_sima)

In [ ]:
if os.path.exists(proj_img_path):
    proj_manual = {}
    proj_manual['std_img'] = np.array(Image.open(proj_img_path))

    manual_data_dims = proj_manual[f'{proj_type}_img'].shape
elif os.path.exists(sima_h5_path):
    # load video data
    # open h5 to read, find data key, grab data, then close
    h5 = h5py.File(sima_h5_path,'r')
    sima_data = np.squeeze(np.array( h5[list(h5.keys())[0]] )).astype('int16') # np.array loads all data into memory
    h5.close()

    proj_manual = {'mean_img': np.mean(sima_data, axis = 0), 
                   'max_img': np.max(sima_data, axis = 0), 
                   'std_img': np.std(sima_data, axis = 0) }

    manual_data_dims = sima_data.shape

In [ ]:
# grab ROI masks from sima (these are probably manually drawn ROIs from imagej)
sima_masks = np.load(sima_mask_path)

if isinstance(rois_to_plot, int): # if int is supplied, first n user-curated rois included in analysis
    rois_to_plot = np.arange(rois_to_plot)
elif rois_to_plot is None: # if None is supplied, all user-curated rois included in analysis
    rois_to_plot = np.arange(sima_masks.shape[0])

num_rois = len(rois_to_plot)

In [ ]:
# define number of ROIs to visualize and make colormap for contours
colors_roi = plt.cm.viridis(np.linspace(0,1,num_rois))

# make list of roi centroids for identifying where to plot each ROI's label on the projection/contour image
roi_label_loc_manual = []
for idx, iROI in enumerate(rois_to_plot):
    
    # make binary map of ROI pixels
    ypix_roi, xpix_roi = np.where(sima_masks[iROI,:,:] == 1)
    
    if ypix_roi.size == 0:
        roi_label_loc_manual.append( [0, 0] )
    else:
        roi_label_loc_manual.append( [np.min(ypix_roi), np.min(xpix_roi)] )


In [ ]:
# plot contours and cell numbers on projection image
clims = [ np.min(proj_manual[f'{proj_type}_img']), 
        np.max(proj_manual[f'{proj_type}_img']) ]

fig, ax = plt.subplots(1, 1, figsize = (10,10))
im = ax.imshow(proj_manual[f'{proj_type}_img'], cmap = 'gray', vmin = clims[0]*1, vmax = clims[1]*0.8)
ax.axis('off')

for iROI, roi_id in enumerate(rois_to_plot): 
    ax.contour(sima_masks[roi_id,:,:], colors=[colors_roi[iROI]])
    ax.text(roi_label_loc_manual[iROI][1]-2, roi_label_loc_manual[iROI][0]-2,  str(roi_id), fontsize=18, weight='bold', color = colors_roi[iROI]);

plt.savefig(os.path.join(fig_save_dir, 'roi_contour_map.jpg'))
plt.savefig(os.path.join(fig_save_dir, 'roi_contour_map.pdf'))

In [ ]:
# initialize variables for plotting time-series
num_samps = signal_dff.shape[-1]
total_time = num_samps/fs 
tvec = np.linspace(0,total_time,num_samps)

In [ ]:
fig, ax = plt.subplots(num_rois, 1, figsize = (9,2*num_rois), sharey=True)
for idx, iROI in enumerate(rois_to_plot):
    
    baseline = np.mean(roi_signal_sima[iROI,:])
    
    to_plot = ( (roi_signal_sima[iROI,:]-baseline)/baseline )*100
    
    ax[idx].plot(tvec, np.transpose( to_plot ), color = colors_roi[idx]);
        
    ax[idx].autoscale(enable=True, axis='x', tight=True) 
    ax[idx].tick_params(axis='both', which='major', labelsize=13)
    ax[idx].tick_params(axis='both', which='minor', labelsize=13)
    
    if idx == np.ceil(num_rois/2-1):
        ax[idx].set_ylabel('Fluorescence Level',fontsize = 20)
    
ax[idx].set_xlabel('Time (s)',fontsize = 20);
    
plt.savefig(os.path.join(fig_save_dir, 'roi_ts.jpg'))
plt.savefig(os.path.join(fig_save_dir, 'roi_ts.pdf'))

